# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [29]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2280060612285,
 'week52high': 145.69,
 'week52low': 53.44,
 'week52highSplitAdjustOnly': 150.66,
 'week52lowSplitAdjustOnly': 53.78,
 'week52change': 0.6827220297800966,
 'sharesOutstanding': 17422807813,
 'float': 0,
 'avg10Volume': 84187931,
 'avg30Volume': 101603760,
 'day200MovingAvg': 127.73,
 'day50MovingAvg': 135.14,
 'employees': 153963,
 'ttmEPS': 3.72,
 'ttmDividendRate': 0.8479286371528713,
 'dividendYield': 0.006328307459936501,
 'nextDividendDate': '0',
 'exDividendDate': '2021-01-30',
 'nextEarningsDate': '2021-01-24',
 'peRatio': 35.25728396479563,
 'beta': 1.1749975138682696,
 'maxChangePercent': 50.62465212343228,
 'year5ChangePercent': 4.851690615655042,
 'year2ChangePercent': 2.16256388104567,
 'year1ChangePercent': 0.639221185480491,
 'ytdChangePercent': -0.020064829072446614,
 'month6ChangePercent': 0.1273403020628746,
 'month3ChangePercent': 0.0977014398859168,
 'month1ChangePercent': 0.01811557120337937,
 'day30ChangePer

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.639221185480491

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [23]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_string)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,130.83,0.50612,N/A
1,AAL,19.00,-0.355621,N/A
2,AAP,172.91,0.197964,N/A
3,AAPL,132.60,0.632429,N/A
4,ABBV,105.86,0.18009,N/A
...,...,...,...,...
500,YUM,107.60,0.019871,N/A
501,ZBH,164.21,-0.0136,N/A
502,ZBRA,505.93,1.044597,N/A
503,ZION,52.70,0.177215,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.29,2.16454,N/A
1,179,FCX,39.32,2.135295,N/A
2,387,PYPL,296.65,1.31679,N/A
3,24,ALGN,607.09,1.179603,N/A
4,275,LB,50.79,1.121018,N/A
5,502,ZBRA,507.48,1.082707,N/A
6,410,SIVB,562.20,1.065972,N/A
7,128,DE,333.00,1.061345,N/A
8,6,ABMD,332.09,0.996051,N/A
9,266,KLAC,335.27,0.984959,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [18]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe

/home/aky/Code/algorithmic-trading-python/env/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.29,2.16454,522
1,179,FCX,39.32,2.135295,508
2,387,PYPL,296.65,1.31679,67
3,24,ALGN,607.09,1.179603,32
4,275,LB,50.79,1.121018,393
5,502,ZBRA,507.48,1.082707,39
6,410,SIVB,562.20,1.065972,35
7,128,DE,333.00,1.061345,60
8,6,ABMD,332.09,0.996051,60
9,266,KLAC,335.27,0.984959,59


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [87]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,127.66,N/A,0.523052,N/A,0.307363,N/A,0.17147,N/A,-0.003863,N/A,N/A
1,AAL,19.48,N/A,-0.342861,N/A,0.474657,N/A,0.48323,N/A,0.17464,N/A,N/A
2,AAP,170.33,N/A,0.200172,N/A,0.037028,N/A,0.140177,N/A,0.000314,N/A,N/A
3,AAPL,135.57,N/A,0.62935,N/A,0.131887,N/A,0.096293,N/A,0.017621,N/A,N/A
4,ABBV,106.56,N/A,0.1826,N/A,0.121421,N/A,0.068013,N/A,-0.064548,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.71,N/A,0.018989,N/A,0.135755,N/A,0.008101,N/A,-0.016926,N/A,N/A
501,ZBH,157.26,N/A,-0.013538,N/A,0.149003,N/A,0.071871,N/A,-0.020795,N/A,N/A
502,ZBRA,512.52,N/A,1.050837,N/A,0.744782,N/A,0.376584,N/A,0.21731,N/A,N/A
503,ZION,53.00,N/A,0.176474,N/A,0.600069,N/A,0.361463,N/A,0.056049,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [88]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0
            
for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,127.66,N/A,0.523052,89.108911,0.307363,63.564356,0.17147,66.138614,-0.003863,34.455446,N/A
1,AAL,19.48,N/A,-0.342861,2.178218,0.474657,80.19802,0.48323,96.039604,0.17464,94.653465,N/A
2,AAP,170.33,N/A,0.200172,64.158416,0.037028,22.178218,0.140177,61.584158,0.000314,37.227723,N/A
3,AAPL,135.57,N/A,0.62935,93.465347,0.131887,41.584158,0.096293,51.287129,0.017621,49.108911,N/A
4,ABBV,106.56,N/A,0.1826,62.574257,0.121421,38.613861,0.068013,45.544554,-0.064548,8.316832,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.71,N/A,0.018989,38.415842,0.135755,42.970297,0.008101,28.712871,-0.016926,24.950495,N/A
501,ZBH,157.26,N/A,-0.013538,32.871287,0.149003,45.148515,0.071871,46.930693,-0.020795,22.574257,N/A
502,ZBRA,512.52,N/A,1.050837,98.811881,0.744782,92.673267,0.376584,91.881188,0.21731,97.821782,N/A
503,ZION,53.00,N/A,0.176474,60.39604,0.600069,87.326733,0.361463,89.90099,0.056049,67.524752,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [89]:
from statistics import mean

for row in hqm_dataframe.index:
    momemtum_percentile = []
    for time_period in time_periods:
        momemtum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momemtum_percentile)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,127.66,N/A,0.523052,89.108911,0.307363,63.564356,0.17147,66.138614,-0.003863,34.455446,63.316832
1,AAL,19.48,N/A,-0.342861,2.178218,0.474657,80.19802,0.48323,96.039604,0.17464,94.653465,68.267327
2,AAP,170.33,N/A,0.200172,64.158416,0.037028,22.178218,0.140177,61.584158,0.000314,37.227723,46.287129
3,AAPL,135.57,N/A,0.62935,93.465347,0.131887,41.584158,0.096293,51.287129,0.017621,49.108911,58.861386
4,ABBV,106.56,N/A,0.1826,62.574257,0.121421,38.613861,0.068013,45.544554,-0.064548,8.316832,38.762376
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.71,N/A,0.018989,38.415842,0.135755,42.970297,0.008101,28.712871,-0.016926,24.950495,33.762376
501,ZBH,157.26,N/A,-0.013538,32.871287,0.149003,45.148515,0.071871,46.930693,-0.020795,22.574257,36.881188
502,ZBRA,512.52,N/A,1.050837,98.811881,0.744782,92.673267,0.376584,91.881188,0.21731,97.821782,95.29703
503,ZION,53.00,N/A,0.176474,60.39604,0.600069,87.326733,0.361463,89.90099,0.056049,67.524752,76.287129


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [90]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,64.76,N/A,0.865138,97.029703,1.390706,99.405941,0.89788,99.405941,0.447173,99.80198,98.910891
1,FCX,37.84,N/A,2.162675,100.0,1.5471,99.60396,0.812853,99.009901,0.189712,96.039604,98.663366
2,DISCA,53.33,N/A,0.673613,95.049505,1.343235,99.207921,1.06758,100.0,0.406759,99.60396,98.465347
3,TWTR,73.93,N/A,0.903113,97.425743,0.862364,96.237624,0.680392,98.415842,0.584679,100.0,98.019802
4,SIVB,543.00,N/A,1.036103,98.613861,1.251244,98.811881,0.533102,97.029703,0.148993,91.485149,96.485149
5,DISCK,42.80,N/A,0.475327,87.920792,1.106277,98.019802,0.936401,99.80198,0.355498,99.405941,96.287129
6,ZBRA,512.52,N/A,1.050837,98.811881,0.744782,92.673267,0.376584,91.881188,0.21731,97.821782,95.29703
7,TPR,40.48,N/A,0.429295,84.752475,1.680652,99.80198,0.394443,92.871287,0.189726,96.237624,93.415842
8,PYPL,297.48,N/A,1.353985,99.60396,0.501419,81.782178,0.526817,96.831683,0.168351,94.059406,93.069307
9,AMAT,120.52,N/A,0.813717,96.633663,0.834501,95.445545,0.57039,97.425743,0.098258,82.772277,93.069307


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [79]:
portfolio_input()

Enter the value of your portfolio:1000000


In [91]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

/home/aky/Code/algorithmic-trading-python/env/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,64.76,308,0.865138,97.029703,1.390706,99.405941,0.89788,99.405941,0.447173,99.80198,98.910891
1,FCX,37.84,528,2.162675,100.0,1.5471,99.60396,0.812853,99.009901,0.189712,96.039604,98.663366
2,DISCA,53.33,375,0.673613,95.049505,1.343235,99.207921,1.06758,100.0,0.406759,99.60396,98.465347
3,TWTR,73.93,270,0.903113,97.425743,0.862364,96.237624,0.680392,98.415842,0.584679,100.0,98.019802
4,SIVB,543.00,36,1.036103,98.613861,1.251244,98.811881,0.533102,97.029703,0.148993,91.485149,96.485149
5,DISCK,42.80,467,0.475327,87.920792,1.106277,98.019802,0.936401,99.80198,0.355498,99.405941,96.287129
6,ZBRA,512.52,39,1.050837,98.811881,0.744782,92.673267,0.376584,91.881188,0.21731,97.821782,95.29703
7,TPR,40.48,494,0.429295,84.752475,1.680652,99.80198,0.394443,92.871287,0.189726,96.237624,93.415842
8,PYPL,297.48,67,1.353985,99.60396,0.501419,81.782178,0.526817,96.831683,0.168351,94.059406,93.069307
9,AMAT,120.52,165,0.813717,96.633663,0.834501,95.445545,0.57039,97.425743,0.098258,82.772277,93.069307


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: